<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/RoBERTa_with_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/8a098b0d-e983-4d24-b69a-755f8ecd253a

In [ ]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

Load data

In [4]:
file_path = './saved_file'
file_path = ('ARPs_and_ProgrammingPosts.xlsx')

data = pd.read_excel(file_path)

# Split the data

In [5]:
# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Load the RoBERTa tokenizer

In [6]:
# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

# Custom Dataset class

In [7]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question_body = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]
        encoding = self.tokenizer.encode_plus(
            question_body,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Split datasets for training and testing

In [8]:
# Create datasets
train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# Training arguments

In [9]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=100,
    save_steps=500,
    save_total_limit=2
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Load the RoBERTa model

In [10]:
# Load the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Trainer

In [11]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Start training
trainer.train()

Step,Training Loss,Validation Loss
100,0.175800,0.151002
200,0.114300,0.126805
300,0.154900,0.099760
400,0.084000,0.141896
500,0.083000,0.105943
600,0.145900,0.133679
700,0.132400,0.098723
800,0.115200,0.120817
900,0.062800,0.105391
1000,0.076100,0.135598


TrainOutput(global_step=2241, training_loss=0.11690260258464609, metrics={'train_runtime': 5913.8124, 'train_samples_per_second': 6.06, 'train_steps_per_second': 0.379, 'total_flos': 9428584668825600.0, 'train_loss': 0.11690260258464609, 'epoch': 3.0})

# Evaluate the model

In [12]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.09864721447229385,
 'eval_runtime': 106.0265,
 'eval_samples_per_second': 28.172,
 'eval_steps_per_second': 1.764,
 'epoch': 3.0}

# Make predictions

In [13]:
# Make predictions
predictions = trainer.predict(test_dataset)
y_true = test_data['Label'].tolist()
y_pred = predictions.predictions.argmax(-1)


# Generate classification report

In [14]:
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1'])
print(report)

              precision    recall  f1-score   support

     Class 0       0.99      0.96      0.98      1504
     Class 1       0.96      0.99      0.98      1483

    accuracy                           0.98      2987
   macro avg       0.98      0.98      0.98      2987
weighted avg       0.98      0.98      0.98      2987

